In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/continuous-grid-arctic/")
sys.path.append("/s/ls4/users/grartem/RL_robots/RL_robotSim/")
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [2]:
os.chdir("/s/ls4/users/grartem/RL_robots/RL_robotSim/")

In [3]:
import time
import gym
import pyhocon
import numpy as np
import ray
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.agents.ddpg.td3 import TD3Trainer

import MyMisc


pygame 2.1.2 (SDL 2.0.16, Python 3.8.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
configs = pyhocon.ConfigFactory.parse_file("/s/ls4/users/grartem/RL_robots/RL_robotSim/Configs/FollowerContinuous/TD3_obst.conf")
CONFIG = configs["td3_explv5"].as_plain_ordered_dict()
CONFIG["config"]["num_workers"]=0

In [5]:
#checkpoint_path = "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst/TD3/explore/td3_explv5/TD3_continuous-grid_bef3b_00001_1_2022-03-14_18-59-52/checkpoint_000320/checkpoint-320"
#table_save_path = "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst/TD3/explore/td3_explv5/TD3_continuous-grid_bef3b_00001_1_2022-03-14_18-59-52/checkpoint-320_test.csv"
checkpoint_path = "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst/TD3/explore/td3_explv5/TD3_continuous-grid_bef3b_00001_1_2022-03-14_18-59-52/checkpoint_000710/checkpoint-710"
table_save_path = "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst/TD3/explore/td3_explv5/TD3_continuous-grid_bef3b_00001_1_2022-03-14_18-59-52/checkpoint-710_test.csv"

In [6]:
#trainer = ray.rllib.agents.registry.get_trainer_class("TD3")(CONFIG["config"])
trainer = TD3Trainer(CONFIG["config"])
trainer.restore(checkpoint_path)

2022-03-18 02:08:15,204	WARNING deprecation.py:45 -- DeprecationWarning: `config['buffer_size']` has been deprecated. Use `config['replay_buffer_config']['capacity']` instead. This will raise an error in the future!
/s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or dire

===Запуск симуляции номер 0===


2022-03-18 02:08:21,486	WARNING deprecation.py:45 -- DeprecationWarning: `convert_to_non_torch_type` has been deprecated. Use `ray/rllib/utils/numpy.py::convert_to_numpy` instead. This will raise an error in the future!
Install gputil for GPU system monitoring.
2022-03-18 02:08:28,022	INFO trainable.py:467 -- Restored on 172.19.12.129 from checkpoint: /s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst/TD3/explore/td3_explv5/TD3_continuous-grid_bef3b_00001_1_2022-03-14_18-59-52/checkpoint_000710/checkpoint-710
2022-03-18 02:08:28,023	INFO trainable.py:475 -- Current state after restoring: {'_iteration': 710, '_timesteps_total': 0, '_time_total': 78126.00439810753, '_episodes_total': 4969}


In [7]:
env = MyMisc.continuous_env_maker(config=CONFIG["config"]["env_config"])

In [ ]:
history = []
for seed_i in range(100):
    print(seed_i)
    env.seed(seed_i)
    obs = env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    start_time = time.time()
    while not done:
        # Compute a single action, given the current observation
        # from the environment.
        action = trainer.compute_single_action(obs, explore=False)
        # Apply the computed action in the environment.
        obs, reward, done, info = env.step(action)
        # Sum up rewards for reporting purposes.
        total_reward += reward
    elapsed_time = time.time() - start_time
    elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    print(elapsed_time)
    history.append({
        "seed": seed_i,
        "reward": total_reward,
        "done": done,
        "time": elapsed_time
    })
    history[-1].update(info)
    # Report results.
pd.DataFrame(history).to_csv(table_save_path, index=False, sep=";", encoding="cp1251")

0
===Запуск симуляции номер 0===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


00:00:56
1
===Запуск симуляции номер 1===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


00:00:59
2
===Запуск симуляции номер 2===


...

Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


00:00:55
20
===Запуск симуляции номер 20===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


00:01:01
21
===Запуск симуляции номер 21===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default
